# Varying slopes and covariance among parameters

The hierarchical linear model of the election forecast is a simple type of mixed-effects model. As a reminder, the model structure was:

$$ \begin{align*}
y_{i, t} &\sim \mathrm{Normal}(\mu_{i, t}, \sigma) &\qquad \mbox{Observation likelihood}\\
\mu_{i, t} &= \alpha + \beta \cdot x_i + \delta_t &\qquad \mbox{Model equation}\\
\alpha &\sim \mathrm{Normal}(0, 0.3) &\qquad \mbox{Prior for overall intercept}\\
\beta_j &\sim \mathrm{Normal}(0, 1) &\qquad \mbox{Prior for slopes}\\
\delta_t &\sim \mathrm{Normal}(0, 0.2) &\qquad \mbox{Prior for varying intercepts} \\
\sigma &\sim \mathrm{HalfCauchy}(2) &\qquad \mbox{Prior for $\sigma$} \end{align*}$$

$i$ is the state index, $t$ is the year index. We have partial pooling across the shifts $\delta_t$ from the hierarchical structure. This is what is called a *varying-intercepts* model, because each group in the population (here, the group is the election year) is allowed to have a different intercept.

# Example

The example we'll look at is the famous UC Berkeley admissions data set.

In [ ]:
# imports
import numpy as np, scipy as sp, pymc3 as pm, matplotlib.pyplot as plt, pandas as pd, theano, theano.tensor as tt
from scipy import stats

In [ ]:
ucbdata = pd.read_csv('ucb.csv', index_col = 0)
ucbdata['male'] = (ucbdata['applicant.gender'] == 'male').astype(int)
ucbdata['dept_id'] = pd.Categorical(ucbdata['dept']).codes
ucbdata

Since the data is binomial, we'll use a logistic regression. The simplest logistic regression we could do is the fully pooled regression:

$$ \begin{align*}
y_{i} &\sim \mathrm{Binomial}(p_{i}, n_i) &\qquad \mbox{Observation likelihood}\\
\mathrm{logit}(p_{i}) &= \alpha + \beta x_i &\qquad \mbox{Model equation}\\
\alpha &\sim \mathrm{Normal}(0, 2) &\qquad \mbox{Prior for varying intercepts}\\
\beta &\sim \mathrm{Normal}(0, 1) &\qquad \mbox{Prior for varying slopes}\\
\sigma &\sim \mathrm{HalfCauchy}(2) &\qquad \mbox{Prior for $\sigma$} \end{align*}$$

Here, $x_i$ is the indicator variable for `male`. Its coefficient would, in principle, measure any apparent gender bias in admission.

To allow for varying slopes as well, we have two options:
* simply generalize by creating and fitting a vector of slopes
* allow the slopes and intercepts to vary *together*, meaning that the parameters are coupled to one another instead of being independent

Let's compare the structure of these models:

$$ \begin{align*}
y_{i} &\sim \mathrm{Binomial}(p_{i}, n_i) &\qquad \mbox{Observation likelihood}\\
\mathrm{logit}(p_{i}) &= \alpha_j + \beta_{j} x_i &\qquad \mbox{Model equation}\\
\alpha_j &\sim \mathrm{Normal}(0, 2) &\qquad \mbox{Prior for varying intercepts}\\
\beta_{j} &\sim \mathrm{Normal}(0, 1) &\qquad \mbox{Prior for varying slopes}\\
\sigma &\sim \mathrm{HalfCauchy}(2) &\qquad \mbox{Prior for $\sigma$} \end{align*}$$

As before, $j$ is the department index, $i$ the observation index, $x_i$ the categorical explanatory variable (1 if male, 0 else).
We could fit this model and call it done.

However, in the real world, slopes and intercepts are often coupled. This is partly because of mathematical reasons: changing the slope of a line typically will change its intercept as well, presuming it still must pass close to the data. It's also because the same causal processes often influence both. To model this, we can still use normal priors for $\alpha_j, \beta_j$, but they must be *multivariate* normals to allow the values to be correlated. So, we have:

$$ \begin{align*}
y_{i} &\sim \mathrm{Binomial}(p_{i}, n_i) &\qquad \mbox{Observation likelihood}\\
\mathrm{logit}(p_{i}) &= \alpha_j + \beta_{j} x_i &\qquad \mbox{Model equation}\\
\left(\begin{array}{c}\alpha_j \\ \beta_j\end{array}\right) &\sim \mathrm{MVNormal}\left(\left(\begin{array}{c}\alpha \\ \beta\end{array}\right), \boldsymbol \Sigma \right) &\qquad \mbox{Prior for coefficients}\\
\alpha &\sim \mathrm{Normal}(0, 2) &\qquad \mbox{Hyperprior for intercepts}\\
\beta &\sim \mathrm{Normal}(0, 1) &\qquad \mbox{Hyperprior for slopes}\\
\boldsymbol \Sigma &\sim ??? &\qquad \mbox{Prior for covariance} \end{align*}
$$

### The covariance matrix

The covariance matrix describes the associations between the various components of a multivariate normal random variable. In our case of a $2\times 2$ matrix, it looks like this:

$$ \boldsymbol\Sigma = \left(\begin{array}{cc} \sigma_a^2 & \sigma_a \sigma_b \rho_{ab} \\ \sigma_a \sigma_b \rho_{ab} & \sigma_b^2 \end{array}\right) $$

This generalizes: for a size-$n$ multivariate normal, the covariance matrix has entries $\boldsymbol \Sigma_{ii} = \sigma^2_{i}, \boldsymbol \Sigma_{ij} = \sigma_i \sigma_j \rho_{ij}$, where $\rho_{ij}$ is the usual Pearson correlation coefficient between variables $i$ and $j$.

### Priors for the covariance matrix

We need a matrix-valued prior for the covariance matrix. Historically, a common choice has been the Wishart distribution (or, inverse-Wishart if we specify the multivariate normal using $\boldsymbol \Sigma^{-1}$ instead -- this is called the *precision matrix*). This is mainly because it is the conjugate prior for a multivariate normal likelihood.

Today, the use of the Wishart prior is discouraged, for both theoretical and practical reasons. The theoretical reasons mostly boil down to the fact that the standard deviations and the correlations are coupled. You can't "pull apart" the standard deviation and correlation, even though they don't need to be associated in the structure of the matrix. The result is that this prior can't be non-informat

The computational reasons are twofold:
* The covariance matrix must be positive definite and symmetric (not too important if you don't know this terminology). These constraints mean that taking a random step from a given covariance matrix will not give a valid covariance matrix (except with probability 0). You can get around this with transformations -- transform to a space without the constraints, sample, and transform back.
* The Wishart distribution has very heavy tails, and MCMC samplers tend to get stuck in them. This means sampling is very inefficient, although it can technically work.

The solution to this is to decouple the individual variances and the correlations, by factoring the covariance matrix as ($2\times 2$ case):
$$
\boldsymbol \Sigma = \left(\begin{array}{cc} \sigma_a & 0 \\ 0 & \sigma_b \end{array} \right) \mathbf{R} \left(\begin{array}{cc} \sigma_a & 0 \\ 0 & \sigma_b \end{array} \right)
$$
where
$$
\mathbf R = \left( \begin{array}{cc} 1 & \rho_{ab} \\ \rho_{ab} & 1 \end{array} \right)
$$
is the correlation matrix. Then we can put separate priors on the correlation matrix and the standard deviations.

Standard deviations are easy -- we've been putting priors on them since early in the semester. The standard choice for the correlation matrix is the LKJ prior, named for Lewandowski, Kurowicka, Joe (2009). This depends on one parameter, $\eta$ (*eta*), which measures a bias toward or away from strong correlations. The distribution is simple:

$$ p(\mathbf{S} | \eta) \propto |\mathbf{S}|^{\eta - 1} $$

This means that $\eta = 1$ is a uniform distribution on correlation matrices; $\eta > 1$ prefers weaker correlations; $\eta < 1$ prefers stronger correlations. So this is a weakly informative, regularizing prior as long as $\eta > 1$.

In [ ]:
# Fully pooled; simple logistic regression
with pm.Model() as simple_model:
    a = pm.Normal('a', 0, 2)
    b = pm.Normal('b', 0, 1)
    sigma = pm.HalfCauchy('sigma', 5)
    p = pm.math.invlogit(a + b * ucbdata.male)
    admit = pm.Binomial('admit', p=p, n=ucbdata.applications, observed = ucbdata.admit)

    trace_simple = pm.sample(2000, target_accept = 0.95, tune = 1000)
    summary_simple = pm.summary(trace_simple)

In [ ]:
pm.forestplot(trace_simple, var_names = ['a', 'b'])

In [ ]:
# Hierarchical, varying intercepts
with pm.Model() as vi_model:
    a = pm.Normal('a', 0, 2)
    b = pm.Normal('b', 0, 1)
    sigma = pm.HalfCauchy('sigma', 5)
    dept_intercept = pm.Normal('dept_intercept', a, sigma, shape=6)
    p = pm.math.invlogit(dept_intercept[ucbdata.dept_id.values] + b * ucbdata.male)
    admit = pm.Binomial('admit', p=p, n=ucbdata.applications, observed = ucbdata.admit)
    
    trace_vi = pm.sample(2000, target_accept = 0.9, tune = 1000)
    summary_vi = pm.summary(trace_vi)

In [ ]:
pm.forestplot(trace_vi, var_names = ['dept_intercept', 'b'])

In [ ]:
# Hierarchical, varying intercepts
with pm.Model() as ind_model:
    a = pm.Normal('a', 0, 2)
    b = pm.Normal('b', 0, 1)
    sigma_a = pm.HalfCauchy('sigma_a', 5)
    sigma_b = pm.HalfCauchy('sigma_b', 5)
    dept_slope = pm.Normal('dept_slope', b, sigma_b, shape = 6)
    dept_intercept = pm.Normal('dept_intercept', a, sigma_a, shape=6)
    p = pm.math.invlogit(dept_intercept[ucbdata.dept_id.values] + dept_slope[ucbdata.dept_id.values] * ucbdata.male)
    admit = pm.Binomial('admit', p=p, n=ucbdata.applications, observed = ucbdata.admit)
    
    trace_ind = pm.sample(2000, target_accept = 0.95, tune = 1000)
    summary_ind = pm.summary(trace_ind, var_names = ['dept_intercept', 'dept_slope'])

In [ ]:
with pm.Model() as vs_model:
    a = pm.Normal('a', 0, 2)
    b = pm.Normal('b', 0, 1)
    
    sd_dist = pm.HalfCauchy.dist(beta = 2)
    
    packed_chol = pm.LKJCholeskyCov('chol_cov', eta = 2, n = 2, sd_dist = sd_dist)
    chol = pm.expand_packed_triangular(2, packed_chol, lower = True)
    cov = tt.dot(chol, chol.T)
    
    sigma_ab = pm.Deterministic('sigma_dept', tt.sqrt(tt.diag(cov)))
    corr = tt.diag(sigma_ab**-1).dot(cov.dot(tt.diag(sigma_ab**-1)))
    r = pm.Deterministic('Rho', corr[np.triu_indices(2, k=1)])
    #chol, corr, sd = pm.LKJCholeskyCov('chol_cov', eta = 2, n = 2, sd_dist = sd_dist, compute_corr = True)#
    mu = pm.MvNormal('ab', mu=tt.stack([a, b]), chol=chol, shape=(6, 2))
    
    dept_intercept = pm.Deterministic('dept_intercept', mu[:, 0])
    dept_slope = pm.Deterministic('dept_slope', mu[:, 1])
    
    p = pm.math.invlogit(mu[ucbdata.dept_id.values, 0] + mu[ucbdata.dept_id.values, 1] * ucbdata.male)
    admit = pm.Binomial('admit', p=p, n=ucbdata.applications, observed=ucbdata.admit)
    
    trace_vs = pm.sample(2000, target_accept = 0.95, tune = 1000)
    summary_vs = pm.summary(trace_vs, var_names = ['dept_intercept', 'dept_slope'])

In [ ]:
pm.forestplot(trace_vs, var_names = ['dept_intercept', 'dept_slope', 'Rho'])

In [ ]:
pm.plot_posterior(trace_vs, var_names = ['Rho'])

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(summary_ind[:6]['mean'], summary_ind[6:]['mean'], 'o', color = 'blue', label = 'independent model')
plt.plot(summary_vs[:6]['mean'], summary_vs[6:]['mean'], 'o', color = 'red', label = 'correlated model')
plt.xlim(-3, 2)
plt.ylim(-1, 0.3)
plt.title('Shrinkage in the model', size = 14)
plt.ylabel('Slope', size = 14)
plt.xlabel('Intercept', size = 14)
for i, txt in enumerate(['A', 'B', 'C', 'D', 'E', 'F']):
    plt.annotate(txt, (summary_ind.iloc[i,0]+0.08, summary_ind.iloc[i+6,0]-0.08))
plt.legend()
plt.show()

In [ ]:
summary_ind

In [ ]:
summary_vs